In [35]:
import requests
from bs4 import BeautifulSoup

root_url = "https://www.immoweb.be/en/search/"
distribution_type = "BUY"
estate_type = "HOUSE"
# need to change in  a list 
# estate_type =['house', 'apartment'] 
country = "BE"

search_url = f"{root_url}{distribution_type}/{estate_type}?countries={country}"
print(search_url)
response = requests.get(search_url)
html_content = response.text
soup = BeautifulSoup(html_content, "html.parser")
house_listings = soup.find_all("a", class_="card__title-link")
house_urls = [listing["href"] for listing in house_listings]
for url in house_urls:
    print(url)


https://www.immoweb.be/en/search/BUY/HOUSE?countries=BE
https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/brakel/9660/10560751
https://www.immoweb.be/en/classified/villa/for-sale/andenne/5300/10561076
https://www.immoweb.be/en/classified/house/for-sale/woluwe-saint-lambert/1200/10560729
https://www.immoweb.be/en/classified/apartment-block/for-sale/genval/1332/10561424
https://www.immoweb.be/en/classified/mixed-use-building/for-sale/antwerp/2060/10559817
https://www.immoweb.be/en/classified/house/for-sale/oupeye/4680/10557728
https://www.immoweb.be/en/classified/house/for-sale/seraing/4100/10557227
https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/geraardsbergen/9500/10557395
https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/wielsbeke/8710/10559981
https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/otegem/8553/10559828
https://www.immoweb.be/en/classified/house/for-sale/anderlecht/